In [1]:
from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta


In [2]:
app = Flask(__name__)

In [6]:
nifty_50_stocks = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'SBIN.NS', 'ITC.NS',
    'TATAMOTORS.NS', 'TATAPOWER.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS'
]


In [7]:
end_date = datetime.today()
start_date = end_date - timedelta(days=30)


In [8]:
stock_data = {}
for symbol in nifty_50_stocks:
    print(f"Downloading data for {symbol}...")
    df = yf.download(symbol, start=start_date, end=end_date, interval='1d', progress=False)
    stock_data[symbol] = df

YF.download() has changed argument auto_adjust default to True


In [9]:
print("\nSample data for RELIANCE.NS:")
print(stock_data['RELIANCE.NS'].tail())


Sample data for RELIANCE.NS:
Price             Close         High          Low         Open      Volume
Ticker      RELIANCE.NS  RELIANCE.NS  RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                      
2025-05-28  1412.900024  1422.599976  1410.699951  1420.099976     5944611
2025-05-29  1417.800049  1420.000000  1408.599976  1412.099976    13771917
2025-05-30  1420.900024  1430.599976  1409.699951  1418.000000    13788484
2025-06-02  1413.699951  1417.500000  1395.599976  1412.099976     8098668
2025-06-03  1405.500000  1426.500000  1403.400024  1422.300049    11252626


In [10]:
{
    "2024-05-01": {
        "RELIANCE.NS": { "Close": 2900.5 },
        "TCS.NS": { "Close": 3500.0 },
        "INFY.NS": { "Close": 1500.0 },
        "SBIN.NS": { "Close": 600.0 },
        "ITC.NS": { "Close": 300.0 },
        "TATAMOTORS.NS": { "Close": 500.0 },
        "TATAPOWER.NS": { "Close": 250.0 },
        "HDFCBANK.NS": { "Close": 1600.0 },
        "ICICIBANK.NS": { "Close": 700.0 },
        "KOTAKBANK.NS": { "Close": 1800.0 }
        
    }
}


{'2024-05-01': {'RELIANCE.NS': {'Close': 2900.5},
  'TCS.NS': {'Close': 3500.0},
  'INFY.NS': {'Close': 1500.0},
  'SBIN.NS': {'Close': 600.0},
  'ITC.NS': {'Close': 300.0},
  'TATAMOTORS.NS': {'Close': 500.0},
  'TATAPOWER.NS': {'Close': 250.0},
  'HDFCBANK.NS': {'Close': 1600.0},
  'ICICIBANK.NS': {'Close': 700.0},
  'KOTAKBANK.NS': {'Close': 1800.0}}}

In [11]:
organized_data = {}

for symbol, df in stock_data.items():
    df.dropna(inplace=True)
    for date, row in df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        if date_str not in organized_data:
            organized_data[date_str] = {}
        organized_data[date_str][symbol] = {
            "Open": round(row['Open'], 2),
            "High": round(row['High'], 2),
            "Low": round(row['Low'], 2),
            "Close": round(row['Close'], 2),
            "Volume": int(row['Volume'])
        }


C:\Users\Priyanshu kumar\AppData\Local\Temp\ipykernel_31064\3546672440.py:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  "Volume": int(row['Volume'])


In [12]:
def calculate_sma(df, sma_length):
    return df['Close'].rolling(window=sma_length).mean()


In [ ]:
sma_length = 10  
sample_df = stock_data['RELIANCE.NS']
sample_df['SMA'] = calculate_sma(sample_df, sma_length)

print(sample_df[['Close', 'SMA']].tail())


Price             Close          SMA
Ticker      RELIANCE.NS             
Date                                
2025-05-28  1412.900024  1431.120007
2025-05-29  1417.800049  1427.480017
2025-05-30  1420.900024  1423.930017
2025-06-02  1413.699951  1421.220007
2025-06-03  1405.500000  1419.310010


In [20]:
from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

app = Flask(__name__)

STOCKS = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'SBIN.NS', 'ITC.NS',
    'TATAMOTORS.NS', 'TATAPOWER.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS'
]

def calculate_sma(df, sma_length):
    return df['Close'].rolling(window=sma_length).mean()

@app.route('/market_insight', methods=['POST'])
def market_insight():
    try:
        data = request.get_json()
        sma_length = int(data.get('sma_length', 10))  

        end_date = datetime.today()
        start_date = end_date - timedelta(days=30)
        
        above_sma = []
        below_sma = []

        for stock in STOCKS:
            df = yf.download(stock, start=start_date, end=end_date, interval='1d', progress=False)
            df.dropna(inplace=True)

            if len(df) < sma_length:
                continue  

            df['SMA'] = calculate_sma(df, sma_length)
            latest = df.iloc[-1]

            
            if latest['Close'] > latest['SMA']:
                above_sma.append(stock)
            else:
                below_sma.append(stock)

        return jsonify({
            "above_sma": above_sma,
            "below_sma": below_sma
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500